## Data Import and Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

from datetime import datetime
import folium
import folium.plugins
from scipy import stats

The Washington DC government posts all of its public Data via the [Open Data Portal](https://opendata.dc.gov/). Parking Enforcement data is available in a variety of formats on a month by month basis. I downloaded that datasets in bulk and stitched them together into the dataset being imported below. I am hosting it for the time being on my personal dropbox. Over time, that may change. [Dataset](https://www.dropbox.com/s/9xzrr4u5n266yrq/Parking%20Violations.csv?dl=1)

In [2]:
data = pd.read_csv('/Users/azel/Documents/Data Sets/DC Data/Parking Enforcement/Combined /Parking Violations.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print()
print(data.info())
print()
print(data.isnull().sum())
print()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14424762 entries, 0 to 14424761
Data columns (total 19 columns):
X                        float64
Y                        float64
OBJECTID                 int64
ROWID_                   float64
DAY_OF_WEEK              object
HOLIDAY                  float64
WEEK_OF_YEAR             float64
MONTH_OF_YEAR            float64
ISSUE_TIME               float64
VIOLATION_CODE           object
VIOLATION_DESCRIPTION    object
LOCATION                 object
RP_PLATE_STATE           object
BODY_STYLE               object
ADDRESS_ID               int64
STREETSEGID              float64
XCOORD                   int64
YCOORD                   int64
TICKET_ISSUE_DATE        object
dtypes: float64(8), int64(4), object(7)
memory usage: 2.0+ GB
None

X                              0
Y                              0
OBJECTID                       0
ROWID_                   2595214
DAY_OF_WEEK              4432946
HOLIDAY                  4432946
WEEK_O

,X,Y,OBJECTID,ROWID_,DAY_OF_WEEK,HOLIDAY,WEEK_OF_YEAR,MONTH_OF_YEAR,ISSUE_TIME,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,BODY_STYLE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE
0,-76.990883,38.901329,24651355,15241.0,THURSDAY,0.0,36.0,9.0,2109.0,P012,DISOBEYING OFFICIAL SIGN,1200 BLOCK I ST NE NORTH SIDE,NY,4D,801937,2090.0,400791,137052,2011-09-01T00:00:00.000Z
1,-77.028853,38.917009,24651356,15242.0,THURSDAY,0.0,36.0,9.0,2158.0,P039,PARK AT EXPIRED METER,1300 BLOCK U ST NW NORTH SIDE,MD,4D,809133,7819.0,397498,138793,2011-09-01T00:00:00.000Z
2,-77.013880,38.888564,24651357,15243.0,THURSDAY,0.0,36.0,9.0,927.0,P036,PARK IN OFFICIAL PARKING PERMIT ONLY SPACE,200 MARYLAND AV SW,DC,TK,801240,1186.0,398796,135635,2011-09-01T00:00:00.000Z
3,-77.054309,38.926666,24651358,15244.0,THURSDAY,0.0,36.0,9.0,107.0,P172,FAILURE TO SECURE DC TAGS,2600 BLOCK GARFIELD ST NW NORTH SID,NJ,4D,806063,5205.0,395291,139866,2011-09-01T00:00:00.000Z
4,-77.023957,38.897877,24651359,15245.0,THURSDAY,0.0,36.0,9.0,1011.0,P199,PARK IN A DESIGNATED ENTRANCE,700 9TH ST NW BLOCK EAST SIDE,DC,NaN,813593,11702.0,397922,136669,2011-09-01T00:00:00.000Z


The first step is to get rid of columns I don't need and rename the remaining columns to a format that is more comfortable to type. Next I work on filling in various NAN  values. The only Violation Description worth filling was the value associated with violation code "PO76". For reference, utlize:  
[DC Muniicipal Regulations Title 18](https://dcregs.dc.gov/Common/DCMR/ChapterList.aspx?TitleNum=18)
[DC Final Adjudication](https://dmv.dc.gov/sites/default/files/dc/sites/dmv/publication/attachments/AdjudicationDispositionRptJan2011.pdf)

Since the dataframe still contains more than 300 unique Violation Codes, the next step is to bin those in to general Violation names.

The final step is to return those Violation Descriptions to the main Dataset.

In [4]:
data = (data.drop(columns = ['OBJECTID','ROWID_','DAY_OF_WEEK','HOLIDAY','WEEK_OF_YEAR','MONTH_OF_YEAR','ISSUE_TIME',
                             'XCOORD','YCOORD','ADDRESS_ID'])
        .rename(columns = {'VIOLATION_DESCRIPTION':'Violation Description','VIOLATION_CODE': 'Violation Code',
                           'STREETSEGID': 'Block ID', 'LOCATION':'Address','RP_PLATE_STATE':'License Plate State', 
                           'BODY_STYLE':'Body Style','TICKET_ISSUE_DATE':'Ticket Issue Date', 'Y': 'Latitude',
                           'X': 'Longitude'}))

na_map = {"P076" : 'ROSA'}
mask = data['Violation Description'].isnull()
data.loc[ mask , 'Violation Description' ] = data.loc[mask, 'Violation Code'].map(na_map)

data['License Plate State'] = data['License Plate State'].fillna('NA')
data['Body Style'] = data['Body Style'].fillna('Other')


In [5]:
meter_violations = ('PARK AT EXPIRED METER','FAIL TO DISPLAY A MULTISPACE METER RECEIPT',
                    'PARK OVERTIME AT A METER','EXPIRATION TIME ON METER RECEIPT LAPSED',
                   'METER, FAIL TO DEPOSIT PAYMENT','EXPIRED HANDICAP METER',
                    'FAIL TO DISPLAY METER RECEIPT IN A LOADING ZONE','NOT PARKED IN A METER SPACE',
                    'OVERSIZED COMM VEHICLE PARKED AT A METER','IND. W/ DISABILITIES METER  NO PROOF OF PAYMENT',
                   'FAIL TO DISPLAY MULTI SPACE RECEIPT BALLPARKEVENT','ILLEGAL AT METER',
                   'FAIL TO DISPLAY MULTI SPACE RECEIPT  BALLPARKEVENT','OVERTIME AT A MULTISPACE METER IN A LOADING ZONE',
                   'FAIL TO PROPERLY DISPLAY MULTISPACE METER RECEIPT','FAIL TO DISPLAY MULTI-SPACE METER RECEIPT PROPERLY',
                   'FAIL TO DISPLAY RESIDENTIAL PARKING PERMIT','FAIL TO DEPOSIT COIN IN METER','OVERSIZED VEHICLE AT METER BALL PARK EVENT')
temp_restrictions = ('NO PARKING STREET CLEANING','NO STOPPING OR STANDING IN PM RUSH HOUR ZONE',
                    'NO STOPPING OR STANDING IN AM RUSH HOUR ZONE','EMERGENCY NO PARKING',
                    'NO PARKING SPECIFIC HOURS','RELOCATE TOW FEE','PARKING ON SNOW EMERGENCY ROUTE WHEN PROHIBITED',
                    'NO PARKING 7 AM TO 6:30 PM')
illegal_location = ('DISOBEYING OFFICIAL SIGN','NO PARKING ANYTIME','UNAUTHORIZED VEHICLE IN LOADING ZONE',
                    'NO STANDING ANYTIME','PARKED WITHIN 25 FEET OF A STOP SIGN','PARK IN A BUS ZONE',
                    'PARK WITHIN 5 FEET OF DRIVEWAY OR ALLEY','VEHICLE ON PRIVATE/PUBLIC PROPERTY WITHOUT CONSENT',
                   'PARK WITHIN 10 FEET OF A FIRE HYDRANT','NO STOPPING ANY TIME',
                   'PARK IN RESERVED SPACE FOR MOTORCYCLES ONLY','PARK ON PUBLIC SPACE','PARK WITH LEFT WHEEL TO THE CURB',
                   'PARK LESS THAN 25 FEET FROM AN INTERSECTION','PARK IN SCHOOL ZONE','PARK LESS THAN 40 FEET FROM AN INTERSECTION',
                    'PARK ON TAXICAB STAND','PARK ON SIDEWALK OR AREA RESERVED FOR PEDESTRIANS','NO STOPPING ANYTIME',
                    'NO STOPPING OR STANDING ANYTIME','GOVERNMENT VEHICLE PARKING ONLY','PARKED ON DESIGNATED VENDOR AREA',
                   'PARK IN RESERVED EMBASSY SPACE','PARK ON OR UNDER AN ELEVATED STRUCTURE','PARK IN A FIRE LANE',
                    'PARK IN MEDIAN, ISLAND, OR SAFETY ZONE','NON-TAXI ON HACK STAND','PARKED IN A RESERVED CAR SHARING SPACE',
                    'OBSTRUCTING ENTRANCE OF PKG GARAGE','ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 1',
                   'PARK ON PRIVATE OR PUBLIC PROPERTY','PARK WITHIN 20 FEET OF A BUS STOP OR ZONE','VALET PARKING/STAGING ZONE',
                   'FAIL TO PARK PARALLEL','VALET PARKING/STAGING ZONE','MOTOR RUNNING UNATTENDED','PARK VEHICLE IN FRONT OF A BARRICADE',
                   'COMMERCIAL MOTOR VEHICLE - NOT OUT OF SERVICE','DANGEROUS VEHICLE ON PRIV/PUBLIC PROP VIOL 1','PARK ON MEDIAN, ISLAND, OR SAFETY ZONE',
                   'OBSTRUCT AN INTERSECTION','PARK BETWEEN ISLAND OR SAFETY ZONE','PARK WITHIN TWENTY FIVE FEET FROM A YIELD SIGN')
obstruct_traffic = ('STAND OR PARK IN ALLEY','STAND OR PARK IN ALLEY','PARK IN A DESIGNATED ENTRANCE','PARK ABREAST OF ANOTHER VEHICLE',
                  'OBSTRUCTING CROSSWALK','STOPPING, STANDING OR PARKING VEHICLE IN BIKE LANE','PARKED IN DRIVEWAY OR ALLEY TO OBSTRUCT SIDEWALK',
                  'PARK MORE THAN 12 INCHES FROM CURB','OPP OR BESIDE CONTRUCTION SITE OBSTRUCTING TRAFFIC','STREETCAR-PARK',
                   'OBSTRUCTING ENTRANCE OF PKG GARAGE, DOOR OR GATE','PARK TO REDUCE ROADWAY TO LESS THAN 10 FEET')
residential_zone = ('RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT','PARK OVERTIME IN TIMED ZONE',
                    'RESIDENTIAL PMT PKG 3RD OFFENSE CALENDAR YEAR','RESIDENTIAL PMT PKG 2ND OFFENSE CALENDAR YEAR',
                   'PARK IN OFFICIAL PARKING PERMIT ONLY SPACE','INDIVID W/ DISABILITIES ONLY UNAUTH. USE RSRVD SP',
                   'RESERVED RESIDENTIAL SPACE PERSON W/ DISABILITIES','RESERVED RESIDENTIAL SPACE PERSONS W/ DISABILTIES')
admin_violation = ('FAILURE TO DISPLAY CURRENT TAGS','NO FRONT TAGS.','FAIL TO REPORT FOR INSPECTION.',
                  'FAIL TO DISPLAY CURRENT INSPECTION STICKERS','NO REAR TAGS.','EXPIRED INSPECTION REJECTION STICKER',
                  'IMPROPER DISPLAY OF TAGS','FAIL TO DISPLAY CURRENT INSPECTION STICKER','OBSTRUCTED TAGS.',
                  'FAIL TO PROPERLY AFFIX VALIDATION STICKERS','FAIL TO REMOVE EXPIRED RESIDENTIAL PARKING PERMIT')
rosa_violation = ('FAILURE TO SECURE DC TAGS','ROSA')

In [6]:
data['Violation Bins'] = (data['Violation Description']   #Replacing values to decrease entry variety
                                      .replace(meter_violations,'Meter Violation')
                                      .replace(temp_restrictions,'Temporary Parking Restriction')
                                      .replace(illegal_location,'Illegal Parking Location')
                                      .replace(residential_zone, 'Residential Zone Violation')
                                      .replace(admin_violation, 'DMV Related Violation')
                                      .replace(rosa_violation, 'ROSA Violation')
                                      .replace(obstruct_traffic, 'Obstructing Traffic'))

In [7]:
ticket_reasons = ('Meter Violation', 'Illegal Parking Location',
       'Temporary Parking Restriction', 'Residential Zone Violation',
       'DMV Related Violation', 'ROSA Violation', 'Obstructing Traffic')

data = data.loc[data['Violation Bins'].isin(ticket_reasons)]

In [8]:
#I want the top 3 states isolated and everything else summed up as 'Other'
replace_plates = data['License Plate State'].value_counts().sort_values().head(130).index
data['License Plate State'] = data['License Plate State'].replace(replace_plates,'Other')

At this point I will drop whatever remains. I have managed to keep 99.5% of the DataFrame intact while making the information easier to work with. 

In [ ]:
data = data.dropna()

In [9]:
data['Ticket Issue Date'] = pd.to_datetime(data['Ticket Issue Date'], infer_datetime_format = True).astype(str)

monthly = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%Y-%m')
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%Y')
weekly = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%Y-%V')

data['Month'] = data['Ticket Issue Date'].map(month)
data['M/Y'] = data['Ticket Issue Date'].map(monthly)
data['Year'] = data['Ticket Issue Date'].map(year)
data['W/Y'] = data['Ticket Issue Date'].map(weekly)

Winter = ('12', '1', '2')
Spring = ('3', '4', '5')
Summer = ('6', '7','8')
Fall = ('9', '10', '11')

data['Season'] = (data['Month'].astype(str)
                .replace(Winter, 'Winter')
                .replace (Spring, 'Spring')
                .replace (Summer, 'Summer') 
                .replace (Fall, 'Fall'))

#data['Month'] = data['Ticket Issue Date'].map(month)

#data['Day of the Week'] = data['Day of the Week'].replace({0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})

In [12]:
df = data

print(df.shape)
print()
print(df.info())
print()
print(df.isnull().sum())
print()
df.head()

(14386868, 15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14386868 entries, 0 to 14424761
Data columns (total 15 columns):
Longitude                float64
Latitude                 float64
Violation Code           object
Violation Description    object
Address                  object
License Plate State      object
Body Style               object
Block ID                 float64
Ticket Issue Date        object
Violation Bins           object
Month                    int64
M/Y                      object
Year                     object
W/Y                      object
Season                   object
dtypes: float64(3), int64(1), object(11)
memory usage: 1.7+ GB
None

Longitude                    0
Latitude                     0
Violation Code               0
Violation Description        0
Address                      0
License Plate State          0
Body Style                   0
Block ID                 32535
Ticket Issue Date            0
Violation Bins               0
Month   

,Longitude,Latitude,Violation Code,Violation Description,Address,License Plate State,Body Style,Block ID,Ticket Issue Date,Violation Bins,Month,M/Y,Year,W/Y,Season
0,-76.990883,38.901329,P012,DISOBEYING OFFICIAL SIGN,1200 BLOCK I ST NE NORTH SIDE,Other,4D,2090.0,2011-09-01 00:00:00,Illegal Parking Location,9,2011-09,2011,2011-35,Fall
1,-77.028853,38.917009,P039,PARK AT EXPIRED METER,1300 BLOCK U ST NW NORTH SIDE,MD,4D,7819.0,2011-09-01 00:00:00,Meter Violation,9,2011-09,2011,2011-35,Fall
2,-77.013880,38.888564,P036,PARK IN OFFICIAL PARKING PERMIT ONLY SPACE,200 MARYLAND AV SW,DC,TK,1186.0,2011-09-01 00:00:00,Residential Zone Violation,9,2011-09,2011,2011-35,Fall
3,-77.054309,38.926666,P172,FAILURE TO SECURE DC TAGS,2600 BLOCK GARFIELD ST NW NORTH SID,Other,4D,5205.0,2011-09-01 00:00:00,ROSA Violation,9,2011-09,2011,2011-35,Fall
4,-77.023957,38.897877,P199,PARK IN A DESIGNATED ENTRANCE,700 9TH ST NW BLOCK EAST SIDE,DC,Other,11702.0,2011-09-01 00:00:00,Obstructing Traffic,9,2011-09,2011,2011-35,Fall


### Tickets by State 

In [13]:
df["License Plate State"].value_counts()

MD       5147480
DC       4091855
VA       3256980
Other    1890553
Name: License Plate State, dtype: int64

In [14]:
round(pd.crosstab(df['License Plate State'], df['Violation Bins'], normalize = 'columns')*100, 2)

Violation Bins,DMV Related Violation,Illegal Parking Location,Meter Violation,Obstructing Traffic,ROSA Violation,Residential Zone Violation,Temporary Parking Restriction
License Plate State,,,,,,,
DC,48.00,31.27,24.08,24.79,0.12,22.60,36.61
MD,15.85,36.47,38.07,47.00,49.52,40.09,30.06
Other,8.56,12.55,11.65,10.60,26.50,13.95,15.01
VA,27.60,19.71,26.20,17.62,23.86,23.36,18.33


In [47]:
round(pd.crosstab(df['License Plate State'], df['Violation Bins'], normalize = 'columns')*100, 2)

Violation Bins,DMV Related Violation,Illegal Parking Location,Meter Violation,Obstructing Traffic,ROSA Violation,Residential Zone Violation,Temporary Parking Restriction
License Plate State,,,,,,,
DC,48.00,31.27,24.08,24.79,0.12,22.60,36.61
MD,15.85,36.47,38.07,47.00,49.52,40.09,30.06
Other,8.56,12.55,11.65,10.60,26.50,13.95,15.01
VA,27.60,19.71,26.20,17.62,23.86,23.36,18.33


In [48]:
round(pd.crosstab(df['License Plate State'], df['Violation Bins'], normalize = True)*100, 2)

Violation Bins,DMV Related Violation,Illegal Parking Location,Meter Violation,Obstructing Traffic,ROSA Violation,Residential Zone Violation,Temporary Parking Restriction
License Plate State,,,,,,,
DC,3.95,7.67,7.33,1.11,0.01,2.48,5.89
MD,1.31,8.95,11.59,2.10,2.60,4.40,4.84
Other,0.71,3.08,3.55,0.47,1.39,1.53,2.42
VA,2.27,4.84,7.98,0.79,1.25,2.56,2.95


In [52]:

md = df.loc[df['License Plate State'] == 'MD']
MDOT = md.loc[md['Violation Bins'] == 'Obstructing Traffic']
MDIP = md.loc[md['Violation Bins'] == 'Illegal Parking Location']


In [46]:
round(pd.crosstab(md['Violation Bins'], md['Year'], normalize = 'columns')*100, 1)

Year,2011,2012,2013,2014,2015,2016,2017,2018
Violation Bins,,,,,,,,
DMV Related Violation,3.0,3.0,3.2,3.7,3.6,4.0,4.5,4.7
Illegal Parking Location,21.6,23.3,24.8,25.8,25.7,26.8,26.7,26.3
Meter Violation,35.9,34.7,33.4,31.9,31.4,29.3,31.3,30.6
Obstructing Traffic,6.8,6.9,6.7,6.2,5.6,5.2,4.8,3.9
ROSA Violation,6.9,6.2,7.4,8.2,7.8,8.5,5.6,7.4
Residential Zone Violation,13.3,13.1,10.8,11.1,12.3,11.8,13.1,12.9
Temporary Parking Restriction,12.4,12.9,13.7,13.2,13.6,14.5,13.9,14.2


In [33]:
MD['Violation Description'].value_counts()

STAND OR PARK IN ALLEY                                101961
PARK ABREAST OF ANOTHER VEHICLE                        96675
PARK IN A DESIGNATED ENTRANCE                          53578
STOPPING, STANDING OR PARKING VEHICLE IN BIKE LANE     18253
OBSTRUCTING CROSSWALK                                  15829
PARKED IN DRIVEWAY OR ALLEY TO OBSTRUCT SIDEWALK        5589
OPP OR BESIDE CONTRUCTION SITE OBSTRUCTING TRAFFIC      3636
PARK MORE THAN 12 INCHES FROM CURB                      2857
OBSTRUCTING ENTRANCE OF PKG GARAGE, DOOR OR GATE        2332
PARK TO REDUCE ROADWAY TO LESS THAN 10 FEET             1281
Name: Violation Description, dtype: int64

In [49]:
round(pd.crosstab(MDOT['Violation Description'], MDOT['Year'], normalize = 'columns', margins = True)* 100, 1)

Year,2011,2012,2013,2014,2015,2016,2017,2018,All
Violation Description,,,,,,,,,
OBSTRUCTING CROSSWALK,4.6,4.4,4.0,5.0,5.6,6.2,6.8,7.9,5.2
"OBSTRUCTING ENTRANCE OF PKG GARAGE, DOOR OR GATE",1.5,0.8,0.8,0.4,0.3,0.4,0.8,1.1,0.8
OPP OR BESIDE CONTRUCTION SITE OBSTRUCTING TRAFFIC,1.4,0.4,2.7,1.2,0.7,1.1,1.6,1.0,1.2
PARK ABREAST OF ANOTHER VEHICLE,39.6,40.5,37.8,32.1,28.9,22.1,19.1,19.3,32.0
PARK IN A DESIGNATED ENTRANCE,17.0,18.0,16.7,17.8,18.9,18.6,17.1,17.4,17.7
PARK MORE THAN 12 INCHES FROM CURB,1.5,0.7,0.7,0.8,0.9,1.0,1.4,0.8,0.9
PARK TO REDUCE ROADWAY TO LESS THAN 10 FEET,0.5,0.5,0.4,0.5,0.3,0.4,0.4,0.3,0.4
PARKED IN DRIVEWAY OR ALLEY TO OBSTRUCT SIDEWALK,1.6,1.8,1.7,1.8,1.8,2.0,2.2,2.1,1.9
STAND OR PARK IN ALLEY,28.7,28.8,29.2,33.5,34.7,40.2,43.0,43.5,33.8


In [53]:
round(pd.crosstab(MDIP['Violation Description'], MDIP['Year'], normalize = 'columns', margins = True)* 100, 1)

Year,2011,2012,2013,2014,2015,2016,2017,2018,All
Violation Description,,,,,,,,,
ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
COMMERCIAL MOTOR VEHICLE - NOT OUT OF SERVICE,0.1,0.0,0.2,0.3,0.1,0.2,0.3,0.3,0.2
DANGEROUS VEHICLE ON PRIV/PUBLIC PROP VIOL 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DISOBEYING OFFICIAL SIGN,13.1,20.6,34.7,32.4,29.6,30.3,29.2,28.9,27.3
FAIL TO PARK PARALLEL,0.1,0.2,0.1,0.0,0.1,0.1,0.1,0.1,0.1
GOVERNMENT VEHICLE PARKING ONLY,0.5,0.5,0.3,0.4,0.6,0.6,0.6,0.7,0.5
MOTOR RUNNING UNATTENDED,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
NO PARKING ANYTIME,20.8,18.4,16.0,16.6,15.0,15.3,15.5,15.9,16.7
NO STANDING ANYTIME,15.3,14.4,12.6,11.5,11.3,10.2,11.1,12.0,12.3
